In [31]:
import random
import bisect
import numpy as np
import matplotlib.pyplot as plt

In [76]:
# Constants
a = 1 # Influence of pheromone
b = 3 # Influence of heuristics
p = 0.5 # Evaporation probability
iterations = 100
Q = 0.0001 # Constant for pheromone updates
ant_count = 400 # Count of ants
graphs = { # Map of (file name -> target node), source node is 0 for all
    'yuzSHP55.aco': 55,
    'yuzSHP95.aco': 95,
    'yuzSHP155.aco': 155
}

In [77]:
# Calculates probabilities of transitioning ant to nodes
def calculate_transition_probabilities(ant, nodes, pheromones):
        probabilities = []
        expr_sum = 0
        for j in range(len(nodes)):
            if (j not in ant) and nodes[ant[-1]][j] != 0:
                expr = pow((pheromones[ant[-1]][j]), a) * pow(1 / (nodes[ant[-1]][j]), b)
                expr_sum += expr
                probabilities.append(expr)
            else:
                probabilities.append(0)
        for j in range(len(nodes)):
            probabilities[j] /= expr_sum
        return probabilities
    
# Selects random index taking into account specified probabilities
def random_index(probabilities):
    values = probabilities[:]
    for i in range(1, len(values)):
        values[i] += values[i-1]
    random_value = random.uniform(0, sum(probabilities))
    return bisect.bisect_left(values, random_value)


def calculate_min_path(nodes, source_node, target_node):
    # Init pheromones
    pheromones = [[0.001 for i in range(len(nodes))] for j in range(len(nodes))]
    # Initialize ants, each ant keeps complete path
    ants = {key: [source_node] for key in range(ant_count)}
    # Minimal length, set to impossible value (-1)
    min_length = -1
    # Path to achieve minimal length
    min_path = None
    
    flag_global = 1
    global_min_length = -1
    for i in range(ant_count):
        path_length = 0
        # Keep adding nodes unless target node is reached
        while ants[i][-1] != target_node:
            # Calculate transition pobabilities for current ant
            probabilities = calculate_transition_probabilities(ants[i], nodes, pheromones)
            # Select random index according to probabilities
            selected_index = random_index(probabilities)
            # In
            path_length += nodes[ants[i][-1]][selected_index]
            # Add 
            ants[i].append(selected_index)

        # In case the ant found a better solution - update
        if min_length == -1 or path_length < min_length:
            min_length = path_length
            min_path = ants[i][:]
         
        # Update pheromones
        delta_t = Q / min_length
        for j in range(1, len(ants[i])):
            pheromones[ants[i][j-1]][ants[i][j]] += delta_t
            
    
        # Evaporate pheromones
        for i in range(len(nodes)):
            for j in range(len(nodes)):
                pheromones[i][j] *= (1 - p)
    return min_path, min_length
        

In [80]:
for file_name in graphs:
    nodes = [[int(i) for i in line.split()] for line in open(file_name)]
    min_path, min_length = calculate_min_path(nodes, source_node=0, target_node=len(nodes) - 1)
    
    print('Shortest path:', min_path, 'Min length:', min_length)

Shortest path: [0, 1, 2, 54] Min length: 3
Shortest path: [0, 1, 3, 5, 94] Min length: 12
Shortest path: [0, 1, 3, 5, 7, 154] Min length: 5
